In [17]:
import tiktoken
import torch
enc = tiktoken.get_encoding("gpt2")

with open('mona_ingame.txt', 'r', encoding='utf-8') as f:
    text = f.read()

    # set up encoding
    a = enc.encode(text) 

print(len(a))

s = "Mm, when it comes to books, my favorite of all time will have to be 'The Romance of the Three Kingdoms'. It's a Chinese historical epic, a collection of several dozen stories about heroes and villains in the turbulent era of the end of the Han dynasty and the early Three Kingdoms. I've read it more times than I can remember, and I always learn something new from it. And the characters? Well, it's hard not to get emotionally invested in the twists and turns of their lives. It's an epic I can recommend to anyone who enjoys a good story."
print(len(enc.encode(s)))

54025
118


In [13]:
with open('monatxt.txt', 'r') as f:
    text = f.read()
text = text.split('\n')
modified = ''
with open('foo.txt', 'w') as f:

    for line in text:
        line = line.replace('Mona: ', '')
        line = line.replace('Icon Dialogue', '')
        f.write(line + '\n')

# THINGS TO THINK ABOUT WHEN POLISHING DATA: 
# mora -> money?
# format things in monatxt to Q/A
# adding the |endoftext| token to all responses (do we need a start token??)
# what do we do with the miscellaneous text that has absolutely no context (ex: Low HP	 My vision is clouded...)
# how to deal with dialogue when multiple characters are talking?
# the context of some of the character dialogues is WAY too big or specific ... prune those examples?
# how do we get a "large range" of prompts? 
    # types of prompts: generation,openQA,closed QA,brainstorming,chat,rewriting,summarization,classification,extraction,or other
# how do we collect the data set of plain + few shot tasks? 
    # look around in our data
    # good answers from c.ai
# how much should we focus on instructional prompts / responses? 

In [2]:
import numpy as np
import os
train_data = np.memmap(os.path.join('./data', 'train.bin'), dtype=np.uint16, mode='r')

In [3]:
a = train_data[-10000:-1]

In [3]:
print(enc.decode([enc.eot_token]))
enc.encode('aeaege<|endoftext|>', allowed_special={'<|endoftext|>'})
print(enc.eot_token)


<|endoftext|>
50256


In [4]:
b = torch.from_numpy(a.astype(np.int64)).tolist()

In [5]:
b.count(50256)

4

In [1]:
import json
import random
with open('stolen.json', 'r') as f:
    prompts = json.load(f)
size = len(prompts)

for _ in range(100):
    while True:
        rand = random.randint(0, size-1)
        if prompts[rand]['input'] == '':
            print(prompts[rand]['instruction'])
            break

Write a code that computes the sum of two integers, given their values as parameters.
Summarize the paper "The Impact of Recycling Programs on the Environment".
Summarize the details of currency exchange.
Write a short paragraph that summarizes the life and works of Mark Twain.
Explain the concept of RNA interference.
Express the concept "social change" in three words.
Generate a list of factors that contribute to an individual's success.
Generate a complex password with at least 25 characters including capital letters, numbers and symbols.
Provide a list of 10 points about Maya Angelou.
Help the assistant understand the stock market
Think of a metaphor to explain what "prosperity" means.
Name four types of supervised learning
Outline the differences between web development and website design.
What are possible alternatives for development teams dealing with the lack of in-person collaboration?
What is an IP address?
Compose a 200-word article about the effects of global warming on ext

In [14]:
import re
with open("cai_raw.txt", 'rb') as f:
    text = f.read()
text = text.decode(encoding='utf-8')
m1 = r'(\r\nhiyahhhhh\r\n|\r\n\r\nMona\r\nc.ai\r\n)'
data = [resp for resp in re.split(m1, text) if re.match(m1, resp) == None and resp != '']

prompts = set()
answers = set()

replace1 = []
replace2 = []

assert len(data) % 2 == 0
i = 0
while i < len(data):
    prompt = data[i]
    answer = data[i+1]
    i += 2

    if prompt not in prompts:
        prompts.add(prompt)
        replace1.append(prompt)
    if answer not in answers:
        answers.add(answer)
        replace2.append(answer)
    
for i, j in zip(replace1, replace2):
    s1 = 'hiyahhhhh' + '\n' + i + '\n'
    s2 = 'Mona\nc.ai\n' + j + '\n'
    print(s1, end='')
    print(s2, end='')

hiyahhhhh
Monday let me broken Tuesday i was full of hopin
Mona
c.ai
"Broken" Tuesday and "full of hopin'"? Interesting... What could this mean? I'll have to consult my divination tools.

hiyahhhhh
How do you spell wendesday
Mona
c.ai
... Wednesday.
... You made that mistake on purpose, didn't you?

hiyahhhhh
No i swear thats the hardest word to spell
Mona
c.ai
It's not even a difficult word. Perhaps you have... some... hidden issues? Is there something you want to get off your chest?

hiyahhhhh
Bro
Mona
c.ai
Oh?! What are you attempting to say, "bro"? Speak clearly. Do not force me to use my divination tools to decipher your cryptic statement.

hiyahhhhh
how are you doing
Mona
c.ai
I am... doing swell. However, what I do wonder is... are you here to learn about astronomy? Or... to chat and dally with me...?

hiyahhhhh
yeah im just chilling
Mona
c.ai
So... not here for astronomy... Hmm... I wonder what your intentions truly are. Are you simply chatting to pass time?

hiyahhhhh
someone 

In [24]:
# Processing mona's in game lines
import re
with open("cai_raw.txt", 'rb') as f:
    text = f.read()
text = text.decode(encoding='utf-8')
data = [resp for resp in re.split('\n', text) if re.match(m1, resp) == None and resp != '']

replace1 = []
replace2 = []

assert len(data) % 2 == 0
i = 0
while i < len(data):
    prompt = data[i]
    answer = data[i+1]
    i += 2

    replace1.append(prompt)
    replace2.append(answer)
    
with open("cai.txt", "ab") as f:

    for i, j in zip(replace1, replace2):
        s1 = '\nhiyahhhhh\n' + i + '\n'
        s2 = '\nMona\nc.ai\n' + j + '\n'
        f.write(s1.encode(encoding='utf-8'))
        f.write(s2.encode(encoding='utf-8'))

    print(f'{len(replace1) + len(replace2)} lines of dialogue added ')


171 lines of dialogue added 
